In [ ]:
## Step 0: Import

import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.metrics import roc_auc_score, average_precision_score
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F


In [ ]:
## Step 1: Prep helper: make patient-level sequences
## We:
## read the CSV,
## choose which columns are model inputs (features),
## forward-fill + impute missing values,
## also create a parallel mask of which values were actually observed (informative missingness),
## store each patient as a dict.




FEATURE_COLS = [
    "HR", "O2Sat", "Temp", "SBP", "MAP", "Resp", "pH",
    "BUN", "WBC", "Platelets", "Magnesium", "Potassium"
]
LABEL_COL = "SepsisLabel"
META_COLS = ["PatientID", "Hour"]


def load_and_normalize(train_csv, val_csv):
    # 1) load both
    df_train = pd.read_csv(train_csv)
    df_val   = pd.read_csv(val_csv)

    # 2) sort
    df_train = df_train.sort_values(["PatientID", "Hour"]).reset_index(drop=True)
    df_val   = df_val.sort_values(["PatientID", "Hour"]).reset_index(drop=True)

    # 3) OPTIONAL: drop Hour == 0
    df_train = df_train[df_train["Hour"] > 0]
    df_val   = df_val[df_val["Hour"] > 0]

    # 4) compute normalization stats **on train only**
    train_means = df_train[FEATURE_COLS].mean()
    train_stds  = df_train[FEATURE_COLS].std().replace(0, 1e-6)

    # 5) apply to both train and val
    df_train[FEATURE_COLS] = (df_train[FEATURE_COLS] - train_means) / train_stds
    df_val[FEATURE_COLS]   = (df_val[FEATURE_COLS] - train_means) / train_stds

    return df_train, df_val, train_means, train_stds

def load_and_normalize_one(csv_path): #just for testing
    df = pd.read_csv(csv_path)
    # drop the bookkeeping hour
    df = df[df["Hour"] > 0].sort_values(["PatientID", "Hour"]).reset_index(drop=True)

    # compute stats on this big file
    means = df[FEATURE_COLS].mean()
    stds  = df[FEATURE_COLS].std().replace(0, 1e-6)

    df[FEATURE_COLS] = (df[FEATURE_COLS] - means) / stds
    return df

def build_patient_sequences_simple(csv_path):
    df = pd.read_csv(csv_path)
    df = df.sort_values(["PatientID", "Hour"]).reset_index(drop=True)

    # Keep only what we care about
    keep_cols = META_COLS + FEATURE_COLS + [LABEL_COL]
    df = df[keep_cols]

    patients = []

    # Precompute global means for fallback impute at the end
    global_means = df[FEATURE_COLS].mean(skipna=True)

    for pid, g in df.groupby("PatientID"):
        g = g.sort_values("Hour")
        
        # Skip the 'bookkeeping' hour
        g = g[g['Hour'] > 0]
        

        # Patient label: septic if ever SepsisLabel==1 i.e. ever septic
        y_patient = int(g[LABEL_COL].fillna(0).max())

        # Grab features only
        X = g[FEATURE_COLS].copy()
        X = X.to_numpy(dtype=float)

        ## 1. forward fill within patient
        ##X = X.ffill()

        ## 2. fill any still-missing with global means
        ##X = X.fillna(global_means)

        # convert to tensor [T, F]
        X_tensor = torch.tensor(X.to_numpy(dtype=np.float32))

        patients.append({
            "patient_id": pid,
            "series": X_tensor,                        # [T, F]
            "label": torch.tensor(y_patient).float(), # scalar
            "length": X_tensor.shape[0],
        })

     # Also return feature size for model building
    in_features = len(FEATURE_COLS)
    return patients, in_features

def df_to_patients(df):
    patients = []
    for pid, g in df.groupby("PatientID"):
        g = g.sort_values("Hour")
        
        # Skip the 'bookkeeping' hour
        g = g[g['Hour'] > 0]
        if g.empty:
            continue
        
        # patient-level label: ever septic
        y = int(g[LABEL_COL].fillna(0).max())

        # take feature columns and fill NaNs
        X = g[FEATURE_COLS].copy()
        X = X.fillna(0.0)   # <— very important. We assume the dataset is processed, this is for debug

        # convert to tensor [T, F]
        X_tensor = torch.tensor(
            X.to_numpy(dtype="float32"),
            dtype=torch.float32,
        )

        patients.append({
            "patient_id": pid,
            "series": X_tensor,          # [T, F]
            "label": torch.tensor(y).float(),
            "length": X_tensor.shape[0],
        })
    in_features = len(FEATURE_COLS)
    return patients, in_features

In [94]:
# Step 2: PyTorch Dataset for patients, This is basically a light wrapper around that list.
class TCNDataset(Dataset):
    def __init__(self, patients_list):
        self.patients = patients_list

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        p = self.patients[idx]
        return {
            "series": p["series"],        # [T, F]
            "label": p["label"],          # scalar
            "length": p["length"],        # int
            "patient_id": p["patient_id"]
        }

In [95]:
## Step 3: Collate function for variable-length padding
## finds the max length in the batch,
## pads everyone to that length,
## builds a mask so we know which timesteps are real,
## permutes to [B, C, T] for TCNs.


def collate_tcn(batch):
    lengths = [b["length"] for b in batch]
    max_len = max(lengths)
    feat_dim = batch[0]["series"].shape[1]
    B = len(batch)

    feats = torch.zeros(B, max_len, feat_dim, dtype=torch.float32)
    time_mask = torch.zeros(B, max_len, dtype=torch.float32)
    labels = torch.zeros(B, dtype=torch.float32)
    last_idx = torch.zeros(B, dtype=torch.long)

    patient_ids = []

    for i, b in enumerate(batch):
        T = b["length"]
        feats[i, :T, :] = b["series"]
        time_mask[i, :T] = 1.0
        labels[i] = b["label"]
        last_idx[i] = T - 1
        patient_ids.append(b["patient_id"])

    # TCN expects [B, C, T]
    feats = feats.permute(0, 2, 1)

    return {
        "x": feats,            # [B, feat_dim, max_len]
        "mask": time_mask,     # [B, max_len] (not strictly needed for patient-level loss)
        "y": labels,           # [B]
        "last_idx": last_idx,  # [B]
        "patient_id": patient_ids
    }

In [96]:
## Step 4: A minimal TCN model

## This is a pretty standard residual dilated 1D-conv block.
## Key details:
## Causal conv (padding = dilation*(kernel_size-1)) so we don’t “peek into the future.”
## Residual connection so gradients flow.

class TemporalBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, dilation, dropout=0.1):
        super().__init__()
        
        # causal padding so output length == input length
        pad = (kernel_size - 1) * dilation

        self.conv1 = nn.Conv1d(in_ch, out_ch,
                               kernel_size=kernel_size,
                               padding=pad,
                               dilation=dilation)
        self.relu1 = nn.ReLU()
        self.drop1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(out_ch, out_ch,
                               kernel_size=kernel_size,
                               padding=pad,
                               dilation=dilation)
        self.relu2 = nn.ReLU()
        self.drop2 = nn.Dropout(dropout)

        # 1x1 conv for residual if channel dims change
        self.residual = (
            nn.Conv1d(in_ch, out_ch, kernel_size=1)
            if in_ch != out_ch else nn.Identity()
        )


        self._init_weights()

    def _init_weights(self):
        for m in [self.conv1, self.conv2]:
            nn.init.kaiming_normal_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        if isinstance(self.residual, nn.Conv1d):
            nn.init.kaiming_normal_(self.residual.weight)
            if self.residual.bias is not None:
                nn.init.zeros_(self.residual.bias)

    def forward(self, x):
        # x: [B, C, T]
        T = x.size(-1)

        y = self.conv1(x)[:, :, :T]
        y = self.relu1(y)
        y = self.drop1(y)

        y = self.conv2(y)[:, :, :T]
        y = self.relu2(y)
        y = self.drop2(y)

        res = self.residual(x)[:, :, :T]

        return y + res  # residual

# Now stack several TemporalBlocks with increasing dilation rates:

class TCNModel(nn.Module):
    def __init__(self, in_ch, hidden_ch=64, num_levels=4, kernel_size=3):
        super().__init__()
        layers = []
        ch_in = in_ch
        for i in range(num_levels):
            dilation = 2 ** i  # 1,2,4,8,...
            ch_out = hidden_ch
            layers.append(
                TemporalBlock(
                    in_ch=ch_in,
                    out_ch=ch_out,
                    kernel_size=kernel_size,
                    dilation=dilation,
                    dropout=0.1,
                )
            )
            ch_in = ch_out

        self.tcn = nn.Sequential(*layers)
        self.head = nn.Conv1d(ch_in, 1, kernel_size=1)  # per-timestep logit

    def forward(self, x):
        # x: [B, in_ch, T]
        h = self.tcn(x)        # [B, hidden_ch, T]
        logits = self.head(h)  # [B, 1, T]
        logits = logits.squeeze(1)  # [B, T]
        return logits

In [97]:
## Step 5: Training step (patient-level label)
## We’ll treat the patient label as “risk at last observed hour.”
## So:
## run model → [B, T]
## pick each patient’s last_idx
## compute sigmoid + BCEWithLogitsLoss


def train_step(model, batch, optimizer, device, pos_weight):
    model.train()
    x = batch["x"].to(device)             # [B, C, T]
    y = batch["y"].to(device)             # [B]
    last_idx = batch["last_idx"].to(device)

    logits_time = model(x)                # [B, T]
    
    # gather last valid timestep logits
    # shape [B]
    B = logits_time.shape[0]
    logits_last = logits_time[torch.arange(B, device=device), last_idx]

    if torch.isnan(logits_last).any():
        print("NaN in logits_last!")
        print("logits_last:", logits_last)
    if torch.isnan(y).any():
        print("NaN in labels!")
    
    # compute class ratio. Use Weighted loss to handle imbalance in neural network
    #num_pos = sum(p["label"].item() for p in train_patients)
    #num_neg = len(train_patients) - num_pos
    #pos_weight = torch.tensor(num_neg / num_pos)
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(device))
    #loss_fn = nn.BCEWithLogitsLoss()
    loss = loss_fn(logits_last, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        probs = torch.sigmoid(logits_last)
        preds = (probs > 0.5).float()
        acc = (preds == y).float().mean()

    return loss.item(), acc.item()

In [98]:
## Evaluation step

@torch.no_grad()
def eval_epoch(model, loader, device):
    model.eval()
    all_probs, all_labels = [], []

    total_loss = 0.0
    count = 0
    loss_fn = nn.BCEWithLogitsLoss()

    for batch in loader:
        x = batch["x"].to(device)
        y = batch["y"].to(device)
        last_idx = batch["last_idx"].to(device)

        logits_time = model(x)             # [B, T]
        B = logits_time.shape[0]
        logits_last = logits_time[torch.arange(B, device=device), last_idx]  # [B]
        loss = loss_fn(logits_last, y)

        total_loss += loss.item()
        count += 1

        probs = torch.sigmoid(logits_last).detach().cpu().numpy()
        labels = y.detach().cpu().numpy()
        all_probs.extend(probs)
        all_labels.extend(labels)

    try:
        auroc = roc_auc_score(all_labels, all_probs)
    except ValueError:
        auroc = float('nan')
    try:
        auprc = average_precision_score(all_labels, all_probs)
    except ValueError:
        auprc = float('nan')

    return {
        "val_loss": total_loss / max(1, count),
        "val_auroc": auroc,
        "val_auprc": auprc,
    }


In [101]:
## Step 6 Putting it all together 
##File path for training data. 

csv_path_train = "~/early-sepsis-prediction/Data/subset_fortesting/subset_test.csv"
csv_path_val_or_test = "~/early-sepsis-prediction/Data/subset_fortesting/subset_val.csv"


train_df, val_df, means, stds = load_and_normalize(csv_path_train, csv_path_val_or_test)

train_patients, in_ch = df_to_patients(train_df)
val_patients, _       = df_to_patients(val_df)

train_dataset = TCNDataset(train_patients)
val_dataset   = TCNDataset(val_patients)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_tcn)
val_loader   = DataLoader(val_dataset,   batch_size=16, shuffle=False, collate_fn=collate_tcn)

## Class imbalance issue: use Weight adjusting to address it
num_pos = sum(int(p["label"].item()) for p in train_patients)
num_neg = len(train_patients) - num_pos

if num_pos == 0:
    pos_weight = torch.tensor([1.0])
else:
    pos_weight = torch.tensor([num_neg / num_pos])

#pos_weight = torch.tensor([pos_weight_value], dtype=torch.float32)
print("pos_weight =", pos_weight)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TCNModel(in_ch=in_ch, hidden_ch=64, num_levels=4, kernel_size=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    # ----- TRAIN -----
    batch_losses = []
    batch_accs = []
    for batch in train_loader:
        loss, acc = train_step(model, batch, optimizer, device, pos_weight)
        batch_losses.append(loss)
        batch_accs.append(acc)
    avg_loss = sum(batch_losses) / len(batch_losses)
    avg_acc  = sum(batch_accs) / len(batch_accs)
    print(f"epoch {epoch} | train_loss={avg_loss:.4f} | train_acc={avg_acc:.4f}")

    # ----- Validation/Test -----
    val_metrics = eval_epoch(model, val_loader, device)
    print(f"             | Val loss={val_metrics['val_loss']:.4f} "
          f"AUROC={val_metrics['val_auroc']:.3f} AUPRC={val_metrics['val_auprc']:.3f}")


pos_weight = tensor([12.3333])
epoch 0 | train_loss=3.0859 | train_acc=0.6667
             | Val loss=0.6355 AUROC=0.396 AUPRC=0.080
epoch 1 | train_loss=1.9570 | train_acc=0.7292
             | Val loss=1.1688 AUROC=0.405 AUPRC=0.080
epoch 2 | train_loss=1.0016 | train_acc=0.5000
             | Val loss=0.8147 AUROC=0.550 AUPRC=0.103
epoch 3 | train_loss=0.5999 | train_acc=0.7500
             | Val loss=0.4267 AUROC=0.631 AUPRC=0.124
epoch 4 | train_loss=0.3023 | train_acc=0.8750
             | Val loss=0.3162 AUROC=0.721 AUPRC=0.166
epoch 5 | train_loss=0.3674 | train_acc=0.9167
             | Val loss=0.3096 AUROC=0.730 AUPRC=0.167
epoch 6 | train_loss=0.3362 | train_acc=0.9375
             | Val loss=0.3108 AUROC=0.739 AUPRC=0.167
epoch 7 | train_loss=0.1683 | train_acc=1.0000
             | Val loss=0.3125 AUROC=0.712 AUPRC=0.155
epoch 8 | train_loss=0.0747 | train_acc=0.9583
             | Val loss=0.3167 AUROC=0.721 AUPRC=0.158
epoch 9 | train_loss=0.0677 | train_acc=0.9792
    

In [84]:
@torch.no_grad()
def eval_epoch(model, loader, device):
    model.eval()
    all_losses = []
    all_accs = []

    all_probs = []
    all_targets = []

    loss_fn = torch.nn.BCEWithLogitsLoss()

    for batch in loader:
        x = batch["x"].to(device)             # [B, C, T]
        y = batch["y"].to(device)             # [B]
        last_idx = batch["last_idx"].to(device)

        logits_time = model(x)                # [B, T]
        B = logits_time.shape[0]
        logits_last = logits_time[torch.arange(B, device=device), last_idx]  # [B]

        loss = loss_fn(logits_last, y)

        probs = torch.sigmoid(logits_last)    # [B]
        preds = (probs > 0.5).float()
        acc = (preds == y).float().mean()

        all_losses.append(loss.item())
        all_accs.append(acc.item())

        all_probs.append(probs.cpu())
        all_targets.append(y.cpu())

    # concat for possible AUROC / etc.
    all_probs = torch.cat(all_probs)      # shape [N_val_patients]
    all_targets = torch.cat(all_targets)  # shape [N_val_patients]

    avg_loss = float(torch.tensor(all_losses).mean())
    avg_acc  = float(torch.tensor(all_accs).mean())

    return {
        "val_loss": avg_loss,
        "val_acc": avg_acc,
        "probs": all_probs,
        "targets": all_targets,
    }